In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection

clip_model = CLIPVisionModelWithProjection.from_pretrained(
    pretrained_model_name_or_path='openai/clip-vit-large-patch14',
    ignore_mismatched_sizes=True
)
clip_model.to(device='cuda:0')

In [ ]:
from transformers import Dinov2Model

dv2_model = Dinov2Model.from_pretrained(
    pretrained_model_name_or_path='facebook/dinov2-base',
    ignore_mismatched_sizes=True
)
dv2_model.to(device='cuda:1')

In [ ]:
from PIL import Image
import src.data.transforms as dT
from src import utils
config = utils.get_config()
wandb_config = utils.load_config('clip.yml')
processor = dT.make_training_processor(config, wandb_config)
image = Image.open('/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/abwao.png')
image = image.convert("RGB")
for _ in range(10):
    inputs = processor.preprocess_image([image])
    print(inputs.shape)

In [ ]:
outputs = clip_model(pixel_values=image.to(device='cuda:0'))
for k, v in outputs.items():
    display(k, v.shape)

In [ ]:
outputs = dv2_model(pixel_values=image.to(device='cuda:1'))
for k, v in outputs.items():
    display(k, v.shape)

In [ ]:
import torch
from torch import nn
from src.models.losses import make_triplet_criterion

input1 = torch.randn(1, 128)
anchors = torch.repeat_interleave(input1, repeats=2, dim=0)
# input2 = torch.randn(1, 128)
positives = torch.repeat_interleave(input1, repeats=2, dim=0)
negatives = torch.repeat_interleave(-input1, repeats=2, dim=0)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
display(1 - cos(anchors, positives))
display(1 - cos(anchors, negatives))
criterion = make_triplet_criterion({'criterion': 'TMWDL-Cosine'})
criterion(anchors, negatives, positives)

In [ ]:
from src.submissions import search
from PIL import Image
from src import utils
import torch

config = utils.get_config()
wandb_run = utils.get_run('2khs9u4f')
model = search.RefConInferenceModel(config, wandb_run, 0)
image = Image.open('/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/abwao.png')
image = image.convert("RGB")
outputs = model(images=image)

del model
torch.cuda.empty_cache()

In [ ]:
from src.submissions import search
from PIL import Image
from src import utils
import torch

config = utils.get_config()
wandb_run = utils.get_run('2khs9u4f')

query_set = search.ImageSet(config, wandb_run, query=True, cuda_idx=0)
corpus_set = search.ImageSet(config, wandb_run, query=False, cuda_idx=0)
query_set.build_embeddings()
corpus_set.build_embeddings()
sbf = search.SearchBruteForce(corpus_set)
sbf.query(query_set)

In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import plotly.express as px

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
# model = AutoModel.from_pretrained('facebook/dinov2-base')
px.imshow(image).show()
inputs = processor(images=image, return_tensors="pt", do_normalize=False)
process_image = inputs['pixel_values'].squeeze().movedim(0, 2).numpy(force=True) * 255

px.imshow(process_image.tolist()).show()
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state


In [ ]:
from typing import Sequence
import torch
import torchvision.transforms.functional as tvF
from torchvision import transforms
from PIL import Image
torch.tensor()
list_image = [Image.open(path)for path in ['/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/ztqsz.png']*2]
tvF.to_tensor(list_image)